<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-러쉬" data-toc-modified-id="1.-러쉬-1">1. 러쉬</a></span></li><li><span><a href="#2.-아로마티카" data-toc-modified-id="2.-아로마티카-2">2. 아로마티카</a></span></li><li><span><a href="#3.-클레어스" data-toc-modified-id="3.-클레어스-3">3. 클레어스</a></span></li><li><span><a href="#4.-이든-미네랄" data-toc-modified-id="4.-이든-미네랄-4">4. 이든 미네랄</a></span></li></ul></div>

오엠, adorable, sephora

(+ 마녀공장, 스킨푸드, 아떼, 마켓컬리, so natural, is green, 오드리앤영)

In [1]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import requests

import warnings
warnings.filterwarnings('ignore')

# 1. 러쉬

In [41]:
url = 'https://lush.co.kr/goods/goods_list.php?cateCd=001019001'
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
# html

In [11]:
# 소분류명 수집
so_cat = html.find_all('span', class_='on')
print(len(so_cat))
so_cat

1


[<span class="on"><a href="?cateCd=001019001">배쓰</a> <em>(72)</em> </span>]

In [17]:
import re

re.compile('[가-힣]+').findall(so_cat[0].text)[0]

'배쓰'

In [4]:
# 상품명 수집
item = html.find_all('span', class_='prdName')
print(len(item))
item[:5]

72


[<span class="prdName">킵 잇 플러피 퍼퓸 버블 보틀</span>,
 <span class="prdName">카마 퍼퓸 버블 보틀</span>,
 <span class="prdName">디어 존 퍼퓸 버블 보틀</span>,
 <span class="prdName">더티 퍼퓸 버블 보틀</span>,
 <span class="prdName">러브 퍼퓸 버블 보틀</span>]

In [6]:
# 가격 수집 (정가)
price = html.find_all('span', class_='cost')
print(len(price))
price[:5]

72


[<span class="cost">
 <strong>￦ 23,000</strong> </span>,
 <span class="cost">
 <strong>￦ 35,000</strong> </span>,
 <span class="cost">
 <strong>￦ 35,000</strong> </span>,
 <span class="cost">
 <strong>￦ 23,000</strong> </span>,
 <span class="cost">
 <strong>￦ 35,000</strong> </span>]

In [9]:
price[0].text.replace('\n','').strip()

'￦ 23,000'

In [10]:
# 러쉬는 할인을 안 해..

In [18]:
# 최종 스크래핑 코드
def cosmetic_scraping(category):
    
    total_data = []
    for cat in category:
        url = f'https://lush.co.kr/goods/goods_list.php?cateCd={cat}'
        html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
        
        # 소분류명, 상품명, 가격 (정가, 할인가)
        so_cat = html.find_all('span', class_='on')
        so_cat2 = re.compile('[가-힣]+').findall(so_cat[0].text)[0]
        item = html.find_all('span', class_='prdName')
        price = html.find_all('span', class_='cost')

        data = []
        for i in range(len(item)):
            item2 = item[i].text
            price2 = price[i].text.replace('\n','').strip()
            data.append([so_cat2, item2, price2])
        total_data.append(data)
    
    df = pd.DataFrame(sum(total_data, []))
    df.columns = ['소분류명', '상품명', '정가']
    return df

In [34]:
# 카테고리 선정 후 수집
df = cosmetic_scraping(['001019001','001019002','001019003','001019004','001019005','001019006','001019007'])
print(df.shape)
df.head()

(352, 3)


,소분류명,상품명,정가
0,배쓰,킵 잇 플러피 퍼퓸 버블 보틀,"￦ 23,000"
1,배쓰,카마 퍼퓸 버블 보틀,"￦ 35,000"
2,배쓰,디어 존 퍼퓸 버블 보틀,"￦ 35,000"
3,배쓰,더티 퍼퓸 버블 보틀,"￦ 23,000"
4,배쓰,러브 퍼퓸 버블 보틀,"￦ 35,000"


In [36]:
df['브랜드명'], df['할인가'] = '러쉬', ' '
df = df[['브랜드명','소분류명','상품명','정가','할인가']]
df['정가'] = df['정가'].apply(lambda x: x.replace('￦ ','') + '원')
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(352, 5)


,브랜드명,소분류명,상품명,정가,할인가
0,러쉬,배쓰,킵 잇 플러피 퍼퓸 버블 보틀,"23,000원",
1,러쉬,배쓰,카마 퍼퓸 버블 보틀,"35,000원",
2,러쉬,배쓰,디어 존 퍼퓸 버블 보틀,"35,000원",
3,러쉬,배쓰,더티 퍼퓸 버블 보틀,"23,000원",
4,러쉬,배쓰,러브 퍼퓸 버블 보틀,"35,000원",


In [37]:
df.to_csv('./cosmetic/러쉬.csv', index=False)

In [39]:
df[df.상품명.duplicated()]

,브랜드명,소분류명,상품명,정가,할인가
54,러쉬,배쓰,레인보우,"20,000원",
71,러쉬,배쓰,더 컴포터,"18,000원",
80,러쉬,샤워,슬리피,"8,000원",
92,러쉬,샤워,레인보우,"20,000원",
93,러쉬,샤워,몬스터스 & 에일리언스,"19,000원",
110,러쉬,보디,팅글,"34,000원",
111,러쉬,보디,팅글 225g,"45,000원",
116,러쉬,보디,오션 솔트 셀프-프리저빙 120g/250g,"27,000원",
131,러쉬,보디,버피 90g,"17,000원",
149,러쉬,페이스,오션 솔트 셀프-프리저빙 120g/250g,"27,000원",


# 2. 아로마티카

In [102]:
url = 'https://aromatica.co.kr/product/list.html?cate_no=681'
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
# html

In [103]:
# 상품명 수집
item = html.find_all('span', style='font-size:15px;color:#000000;')
print(len(item))
item[:5]

51


[<span style="font-size:15px;color:#000000;">티트리 밸런싱 클렌징 바</span>,
 <span style="font-size:15px;color:#000000;">티트리 포어 퓨리파잉 클렌징 워터<br/>1% 나이아신아마이드+0.5% BHA</span>,
 <span style="font-size:15px;color:#000000;">티트리 포어 퓨리파잉 클렌징 워터 리필<br/>1% 나이아신아마이드+0.5% BHA 300ml</span>,
 <span style="font-size:15px;color:#000000;">알로에 하이펙티브 클렌저5% Hyaluronic sol.+ 0.1% Enzyme</span>,
 <span style="font-size:15px;color:#000000;">바이탈라이징 로즈마리 올인원 워시</span>]

In [139]:
# 가격 수집 (정가, 할인가)
price = html.find_all('ul', class_='xans-element- xans-product xans-product-listitem')
print(len(price))
price[0]

51


<ul class="xans-element- xans-product xans-product-listitem"><li class=" xans-record-">
<strong class="title displaynone"><span style="font-size:13px;color:#000000;">판매가</span> </strong> <span style="font-size:13px;color:#000000;text-decoration:line-through;">15,000 won</span><span id="span_product_tax_type_text" style="text-decoration:line-through;"> </span></li>
<li class=" xans-record-">
<strong class="title displaynone"><span style="font-size:15px;color:#44918c;font-weight:bold;">혜택가</span> </strong> <span style="font-size:15px;color:#44918c;font-weight:bold;">9,000 won <span style="font-size:12px;color:#797979;">(40%↓)</span></span></li>
</ul>

In [155]:
if price[2].find_all('span', style='font-size:15px;color:#44918c;font-weight:bold;') == []:
    print(price[2].find_all('span', style='font-size:13px;color:#000000;')[1].text)
else:
    print('할인가:', price[2].find_all('span', style='font-size:15px;color:#44918c;font-weight:bold;')[1].text)
    print('원가:', price[2].find('span', style='font-size:13px;color:#000000;text-decoration:line-through;').text)

할인가: 12,000 won (40%↓)
원가: 20,000 won


In [156]:
# 최종 스크래핑 코드
def cosmetic_scraping(category):
    
    total_data = []
    for cat in category:
        url = f'https://aromatica.co.kr/product/list.html?cate_no={cat}'
        html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
        
        # 소분류명, 상품명, 가격 (정가, 할인가)
        cat_dict = {'681':'스킨 케어','682':'헤어 케어','668':'바디 케어',\
                    '766':'메이크업','696':'라이프 케어','565':'아로마테라피'}
        so_cat = cat_dict[cat]
        item = html.find_all('span', style='font-size:15px;color:#000000;')
        price = html.find_all('ul', class_='xans-element- xans-product xans-product-listitem')

        data = []
        for i in range(len(item)):
            item2 = item[i].text
            if price[i].find_all('span', style='font-size:15px;color:#44918c;font-weight:bold;') == []:
                price2 = price[i].find_all('span', style='font-size:13px;color:#000000;')[1].text
                price3 = ' '
            else:
                price2 = price[i].find('span', style='font-size:13px;color:#000000;text-decoration:line-through;').text
                price3 = price[i].find_all('span', style='font-size:15px;color:#44918c;font-weight:bold;')[1].text
            data.append([so_cat, item2, price2, price3])
        total_data.append(data)
    
    df = pd.DataFrame(sum(total_data, []))
    df.columns = ['소분류명', '상품명', '정가', '할인가']
    return df

In [157]:
# 카테고리 선정 후 수집
category = ['681','682','668','766','696','565']
df = cosmetic_scraping(category)
print(df.shape)
df.head()

(148, 4)


,소분류명,상품명,정가,할인가
0,스킨 케어,티트리 밸런싱 클렌징 바,"15,000 won","9,000 won (40%↓)"
1,스킨 케어,티트리 포어 퓨리파잉 클렌징 워터1% 나이아신아마이드+0.5% BHA,"22,000 won","13,200 won (40%↓)"
2,스킨 케어,티트리 포어 퓨리파잉 클렌징 워터 리필1% 나이아신아마이드+0.5% BHA 300ml,"20,000 won","12,000 won (40%↓)"
3,스킨 케어,알로에 하이펙티브 클렌저5% Hyaluronic sol.+ 0.1% Enzyme,"16,000 won","9,600 won (40%↓)"
4,스킨 케어,바이탈라이징 로즈마리 올인원 워시,"18,000 won","10,800 won (40%↓)"


In [171]:
df['브랜드명'] = '아로마티카'

df['정가'] = df['정가'].apply(lambda x: x.split(' ')[0]+'원')
for i in range(len(df)):
    if df['할인가'][i] != ' ':
        df['할인가'][i] = df['할인가'][i].split(' ')[0]+'원'

df = df[['브랜드명','소분류명','상품명','정가','할인가']]
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(148, 5)


,브랜드명,소분류명,상품명,정가,할인가
0,아로마티카,스킨 케어,티트리 밸런싱 클렌징 바,"15,000원","9,000원"
1,아로마티카,스킨 케어,티트리 포어 퓨리파잉 클렌징 워터1% 나이아신아마이드+0.5% BHA,"22,000원","13,200원"
2,아로마티카,스킨 케어,티트리 포어 퓨리파잉 클렌징 워터 리필1% 나이아신아마이드+0.5% BHA 300ml,"20,000원","12,000원"
3,아로마티카,스킨 케어,알로에 하이펙티브 클렌저5% Hyaluronic sol.+ 0.1% Enzyme,"16,000원","9,600원"
4,아로마티카,스킨 케어,바이탈라이징 로즈마리 올인원 워시,"18,000원","10,800원"


In [174]:
df.to_csv('./cosmetic/아로마티카.csv', index=False)

In [175]:
df[df.상품명.duplicated()]

,브랜드명,소분류명,상품명,정가,할인가
84,아로마티카,바디 케어,티트리 밸런싱 클렌징 바,"15,000원","9,000원"
85,아로마티카,바디 케어,바이탈라이징 로즈마리 올인원 워시,"18,000원","10,800원"
86,아로마티카,바디 케어,바이탈라이징 로즈마리 올인원 워시 리필 500ml,"22,000원","13,200원"
89,아로마티카,바디 케어,바이탈라이징 로즈마리 올인원 로션 300ml,"22,000원","13,200원"
90,아로마티카,바디 케어,바이탈라이징 로즈마리 올인원 로션 리필 500ml,"28,000원","16,800원"
97,아로마티카,바디 케어,나무 돌고래 페이스 & 바디 마사지툴,"19,000원",
111,아로마티카,라이프 케어,아로마티카 에이콜로지 브랜드 북 세트,"28,000원","16,800원"
117,아로마티카,라이프 케어,에센셜 핸드 새니타이저겔 500ml,"12,000원","7,200원"
118,아로마티카,라이프 케어,에센셜 핸드 새니타이저겔 30ml,"3,000원",
123,아로마티카,라이프 케어,퓨어 앤 소프트 여성청결제 300ml,"22,000원","13,200원"


# 3. 클레어스

In [3]:
url = 'https://www.klairs.com/shop/search_result.php?search_str=%EB%B9%84%EA%B1%B4&cate=1030&ctype=big'
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
# html

In [4]:
# 소분류명 수집
so_cat = html.find_all('a', class_='selected')
print(len(so_cat))
so_cat

2


[<a class="selected" href="?search_str=%EB%B9%84%EA%B1%B4&amp;cate=1030&amp;ctype=big">클렌징</a>,
 <a class="selected" href="https://www.klairs.com/shop/search_result.php?search_str=%EB%B9%84%EA%B1%B4&amp;cate=1030&amp;ctype=big">ALL</a>]

In [5]:
so_cat[0].text

'클렌징'

In [36]:
# 상품명 수집
item = html.find_all('p', class_='name')
print(len(item))
item[:3]

11


[<p class="name"><a href="https://www.klairs.com/shop/detail.php?pno=1679091C5A880FAF6FB5E6087EB1B2DC&amp;rURL=https%3A%2F%2Fwww.klairs.com%2Fshop%2Fsearch_result.php%3Fsearch_str%3D%25EB%25B9%2584%25EA%25B1%25B4%26cate%3D1030%26ctype%3Dbig&amp;ctype=1&amp;cno1=">리치 모이스트 포밍 클렌저</a></p>,
 <p class="name"><a href="https://www.klairs.com/shop/detail.php?pno=6C3CF77D52820CD0FE646D38BC2145CA&amp;rURL=https%3A%2F%2Fwww.klairs.com%2Fshop%2Fsearch_result.php%3Fsearch_str%3D%25EB%25B9%2584%25EA%25B1%25B4%26cate%3D1030%26ctype%3Dbig&amp;ctype=1&amp;cno1=">젠틀 블랙 프레쉬 클렌징 오일</a></p>,
 <p class="name"><a href="https://www.klairs.com/shop/detail.php?pno=F718499C1C8CEF6730F9FD03C8125CAB&amp;rURL=https%3A%2F%2Fwww.klairs.com%2Fshop%2Fsearch_result.php%3Fsearch_str%3D%25EB%25B9%2584%25EA%25B1%25B4%26cate%3D1030%26ctype%3Dbig&amp;ctype=1&amp;cno1=">젠틀 블랙 딥 클렌징 오일</a></p>]

In [39]:
# 가격 수집 (정가, 할인가)
price = html.find_all('p', class_='price')
print(len(price))
price[:5]

11


[<p class="price">
 <span class="sprice">12,500</span>원
 		
 	</p>,
 <p class="price">
 <span class="sprice">16,900</span>원
 		
 	</p>,
 <p class="price">
 <span class="sprice">14,900</span>원
 		
 	</p>,
 <p class="price">
 <span class="sale">10%</span>
 <span class="sprice">22,800</span>원
 		<span class="nprice">25,400</span>
 </p>,
 <p class="price">
 <span class="sale">10%</span>
 <span class="sprice">39,500</span>원
 		<span class="nprice">43,900</span>
 </p>]

In [40]:
if price[3].find('span', class_='sale') is None:
    print(price[3].text.strip())
else:
    print('할인가:', price[3].find('span', class_='sprice').text)
    print('원가:', price[3].find('span', class_='nprice').text + '원')

할인가: 22,800
원가: 25,400원


In [46]:
# 최종 스크래핑 코드
def cosmetic_scraping(category):
    
    total_data = []
    for cat in category:
        url = f'https://www.klairs.com/shop/search_result.php?search_str=%EB%B9%84%EA%B1%B4&cate={cat}&ctype=big'
        html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
        
        # 소분류명, 상품명, 가격 (정가, 할인가)
        so_cat = html.find_all('a', class_='selected')[0].text
        item = html.find_all('p', class_='name')
        price = html.find_all('p', class_='price')

        data = []
        for i in range(len(item)):
            item2 = item[i].text
            if price[i].find('span', class_='sale') is None:
                price2 = price[i].text.strip()
                price3 = ' '
            else:
                price2 = price[i].find('span', class_='nprice').text + '원'
                price3 = price[i].find('span', class_='sprice').text + '원'
            data.append([so_cat, item2, price2, price3])
        total_data.append(data)
    
    df = pd.DataFrame(sum(total_data, []))
    df.columns = ['소분류명', '상품명', '정가', '할인가']
    return df

In [47]:
# 카테고리 선정 후 수집
df = cosmetic_scraping(['1030','1005','1024','1025','1028'])
print(df.shape)
df.head()

(36, 4)


,소분류명,상품명,정가,할인가
0,클렌징,리치 모이스트 포밍 클렌저,"12,500원",
1,클렌징,젠틀 블랙 프레쉬 클렌징 오일,"16,900원",
2,클렌징,젠틀 블랙 딥 클렌징 오일,"14,900원",
3,클렌징,GENTLE BLACK duo,"25,400원","22,800원"
4,클렌징,GENTLE BLACK BOX,"43,900원","39,500원"


In [49]:
df['브랜드명'] = '클레어스'
df = df[['브랜드명','소분류명','상품명','정가','할인가']]
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(36, 5)


,브랜드명,소분류명,상품명,정가,할인가
0,클레어스,클렌징,리치 모이스트 포밍 클렌저,"12,500원",
1,클레어스,클렌징,젠틀 블랙 프레쉬 클렌징 오일,"16,900원",
2,클레어스,클렌징,젠틀 블랙 딥 클렌징 오일,"14,900원",
3,클레어스,클렌징,GENTLE BLACK duo,"25,400원","22,800원"
4,클레어스,클렌징,GENTLE BLACK BOX,"43,900원","39,500원"


In [50]:
df.to_csv('./cosmetic/클레어스.csv', index=False)

In [51]:
df[df.상품명.duplicated()]

,브랜드명,소분류명,상품명,정가,할인가


# 4. 이든 미네랄

In [52]:
url = 'https://idunminerals.co.kr/category/face/29/'
html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
# html

In [55]:
# 상품명 수집
item = html.find_all('strong', class_='name')
print(len(item))
item[:3]

12


[<strong class="name"><a class="" href="/product/아이리스-페이스-프라이머/9/category/29/display/1/"><span class="title displaynone"><span style="font-size:15px;color:#282623;">상품명</span> :</span> <span style="font-size:15px;color:#282623;">아이리스 페이스 프라이머</span></a></strong>,
 <strong class="name"><a class="" href="/product/파우더-파운데이션/13/category/29/display/1/"><span class="title displaynone"><span style="font-size:15px;color:#282623;">상품명</span> :</span> <span style="font-size:15px;color:#282623;">파우더 파운데이션</span></a></strong>,
 <strong class="name"><a class="" href="/product/미네랄-브론저/18/category/29/display/1/"><span class="title displaynone"><span style="font-size:15px;color:#282623;">상품명</span> :</span> <span style="font-size:15px;color:#282623;">미네랄 브론저</span></a></strong>]

In [58]:
item[0].text.replace('상품명 : ', '')

'아이리스 페이스 프라이머'

In [83]:
# 가격 수집 (정가, 할인가)
price = html.find_all('ul', class_='xans-element- xans-product xans-product-listitem spec')
print(len(price))
price[:2]

12


[<ul class="xans-element- xans-product xans-product-listitem spec"><li class=" display판매가 xans-record-">
 <strong class="title displaynone"><span style="font-size:14px;color:#282623;">판매가</span> :</strong> <span style="font-size:14px;color:#282623;">42,000원</span><span id="span_product_tax_type_text" style=""> </span></li>
 </ul>,
 <ul class="xans-element- xans-product xans-product-listitem spec"><li class=" display판매가 xans-record-">
 <strong class="title displaynone"><span style="font-size:14px;color:#282623;">판매가</span> :</strong> <span style="font-size:14px;color:#282623;">48,000원</span><span id="span_product_tax_type_text" style=""> </span></li>
 </ul>]

In [86]:
if price[3].find('li', class_=' display할인판매가 xans-record-') is None:
    print(price[3].find('li', class_=' display판매가 xans-record-').text.replace('판매가 : ', '').strip())
else:
    print('할인가:', price[3].find('li', class_=' display할인판매가 xans-record-').text.replace('할인판매가 : ', '').strip())
    print('원가:', price[3].find('li', class_=' display판매가 xans-record-').text.replace('판매가 : ', '').strip())

할인가: 65,600원
원가: 82,000원


In [94]:
# 최종 스크래핑 코드
def cosmetic_scraping(category, number):
    
    total_data = []
    for cat, num in zip(category, number):
        url = f'https://idunminerals.co.kr/category/{cat}/{num}/'
        html = BeautifulSoup(requests.get(url, headers={'User-agent': 'Mozilla/5.0'}).text, "lxml")
        
        # 소분류명, 상품명, 가격 (정가, 할인가)
        so_cat = cat
        item = html.find_all('strong', class_='name')
        price = html.find_all('ul', class_='xans-element- xans-product xans-product-listitem spec')

        data = []
        for i in range(len(item)):
            item2 = item[i].text.replace('상품명 : ', '')
            price2 = price[i].find('li', class_=' display판매가 xans-record-').text.replace('판매가 : ', '').strip()
            if price[i].find('li', class_=' display할인판매가 xans-record-') is None:
                price3 = ' '
            else:
                price3 = price[i].find('li', class_=' display할인판매가 xans-record-').text.replace('할인판매가 : ', '').strip()
            data.append([so_cat, item2, price2, price3])
        total_data.append(data)
    
    df = pd.DataFrame(sum(total_data, []))
    df.columns = ['소분류명', '상품명', '정가', '할인가']
    return df

In [98]:
# 카테고리 선정 후 수집
category = ['프라이머','파운데이션','블러셔','컨투어링','파우더',\
            '마스카라','아이섀도우','페이스','아이','cleansing']
number = ['52','54','56','57','58','64','65','68','69','32']
df = cosmetic_scraping(category, number)
print(df.shape)
df.head()

(26, 4)


,소분류명,상품명,정가,할인가
0,프라이머,아이리스 페이스 프라이머,"42,000원",
1,파운데이션,롱웨어 리퀴드 파운데이션 노르딕 베일,"48,000원",
2,파운데이션,모이스처 리퀴드 파운데이션 노르스켄,"48,000원",
3,파운데이션,파우더 파운데이션,"48,000원",
4,블러셔,미네랄 블러셔 & 브러쉬 세트,"66,000원","52,800원"


In [99]:
df['브랜드명'] = '이든 미네랄'
df = df[['브랜드명','소분류명','상품명','정가','할인가']]
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(26, 5)


,브랜드명,소분류명,상품명,정가,할인가
0,이든 미네랄,프라이머,아이리스 페이스 프라이머,"42,000원",
1,이든 미네랄,파운데이션,롱웨어 리퀴드 파운데이션 노르딕 베일,"48,000원",
2,이든 미네랄,파운데이션,모이스처 리퀴드 파운데이션 노르스켄,"48,000원",
3,이든 미네랄,파운데이션,파우더 파운데이션,"48,000원",
4,이든 미네랄,블러셔,미네랄 블러셔 & 브러쉬 세트,"66,000원","52,800원"


In [100]:
df.to_csv('./cosmetic/이든 미네랄.csv', index=False)

In [101]:
df[df.상품명.duplicated()]

,브랜드명,소분류명,상품명,정가,할인가
